In [12]:
%matplotlib widget 
# global import
import numpy as np
import pandas as pd
from tqdm import tqdm

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
# local import
import lppydsmc as ld

# Explanation
The goal is to refined the model and make sure I am not doing anything stupid.
We'll use bokeh to plot. It will much easier + play with the params.


## Model 
The modeled system is a 2D square of size $0.001$ $m$. We suppose the problem is 0D (no dependance on space).

To simplify things, let's attribute numbers to walls (boundaries): 
- left wall : $b_0$
- top wall : $b_1$
- right wall : $b_2$
- bottom wall : $b_3$

We inject a Maxwellian of $300$ $K$ of ions trough $b_0$. The density of the injection can vary.

Once injected, the ions can collide with the boundaries $b_{1,2,3}$ and each ion has a probability of $p$ to be neutralized when colliding. The boundaries $b_{1,2,3}$ are considered purely diffusive.

Each species can leave the system trough $b_1$. 

We will take Iodine (not that it matter really). Its mass is $2.16 \times 10^{-25}$ $kg$. This yields $\bar v \approx 200$ $m.s^{-1}$.

We thus have the following equations where $N_{I}(t)$ and $N_{I^-}(t)$ are the number of particles in the system :

- $\frac{dN_{I^-}}{dt} = \Gamma_{injection, I^-} S_{b_1} - \Gamma_{out, I^-} S_{b_1} - \Gamma_{neutralization, I^-} S_{b_{1,2,3}}$
- $\frac{dN_{I}}{dt} = - \Gamma_{out, I} S_{b_1} + \Gamma_{neutralization, I^-} S_{b_{1,2,3}}$

We suppose that we are at thermal equilibrium : each species is a the same temperature and we can compute the flux reaching each boundary for each species, it would be :
$\Gamma = \frac{1}{4} n \bar v$, where $n = \frac{N}{V}$ is the density we consider.

Thus the previous system yields :
- $\frac{dN_{I^-}}{dt} = \frac{1}{4} (n_{injection, I^-} - n_{I^-} (1+3p)) \bar v$, where $p$ is the probability of neutralization when colliding with boundaries.
- $\frac{dN_{I}}{dt} = \frac{1}{4} (3 n_{I^-}p - n_I) \bar v$

Which is a system we can easily solve and can be written : $X' = AX + B$, $X = (N_{neutrals}, N_{ions})$.

Where $A = ((,),(,))

In [84]:
# Fixed params
temperature = 300 # K
mass = 2.16e-25 # kg
v_mean = ld.utils.physics.maxwellian_mean_speed(temperature, mass)

dx = dy = dz = 1e-3 # volume system
dS = dy*dz # surface 1 plan
dV = dx*dy*dz

In [95]:
class Model:    
    def __init__(self, **kwargs):
        self.temperature = 300 if 'temperature' not in kwargs else kwargs['temperature']
        self.mass = 2.16e-25 if 'mass' not in kwargs else kwargs['mass']
        self.v_mean = ld.utils.physics.maxwellian_mean_speed(self.temperature, self.mass)
        self.n_walls = 3 if 'n_walls' not in kwargs else kwargs['n_walls']# number of walls with which to collide for neutralization
        self.dx = 1e-3 if 'dx' not in kwargs else kwargs['dx']
        self.dy = 1e-3 if 'dy' not in kwargs else kwargs['dy']
        self.dz = 1e-3 if 'dz' not in kwargs else kwargs['dz']
        self.surface_injection = self.dy*self.dz
        self.volume = self.dx*self.dy*self.dz
        
    
    def integrate(self, x0, density, proba, time_step = 1e-6, iterations = 1000, disable = False):
        self.density = density
        self.proba = proba
        self.time_step = time_step
        self.iterations = iterations
        self.x0 = x0
        
        self.a = self.A(proba)
        self.b = self.B(density)
        
        x = x0
        self.x_arr = np.zeros((iterations+1,2))
        self.x_arr[0,:] = x0
        for iteration in tqdm(range(1,iterations+1), disable = disable):
            # euler explicit
            x += (self.a.dot(x)+self.b)*time_step
            self.x_arr[iteration,:] = x
        return self.x_arr
    
    def A(self, proba):
        cte = 0.25*self.v_mean*self.surface_injection/self.volume # S/V = 1/dx
        loss_neutrals = -cte # neutrals -> neutrals
        loss_ions = -cte
        rate_ions_neutrals = proba * self.n_walls * cte

        return np.array([[ loss_neutrals , rate_ions_neutrals ],
                         [ 0. , loss_ions - rate_ions_neutrals ]])
    
    def B(self, density):
        return np.array([0., 1/4. * density * self.v_mean * self.surface_injection])
    
    def plot_solution(self):
        fig, ax = plt.subplots(constrained_layout = True)
        times = np.arange(self.iterations+1)*self.time_step
        ax.plot(times, self.x_arr[:,0]/self.volume, label = 'Neutral', color = 'r')
        ax.plot(times, self.x_arr[:,1]/self.volume, label = 'Ion', color = 'b')
        ax.legend(loc='best');
        print('number of walls x proba = {:.3f}'.format(self.x_arr[-1,0]/self.x_arr[-1,1]))
        

In [96]:
model = Model(n_walls = 3)

In [99]:
volume = 1e-9
n_init = 1.6e20*volume*0
results = model.integrate(np.array([n_init,n_init]), density = 3.2e20, proba = 0.3, time_step = 1e-7, iterations = 1000)

100%|██████████| 1000/1000 [00:00<00:00, 285055.32it/s]


In [100]:
model.plot_solution()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

number of walls x proba = 0.893


In [ ]:
time_step = 1e-6
volume = 1e-9
iterations = 1000
fig, ax = plt.subplots(constrained_layout = True)
times = np.arange(iterations+1)*time_step
ax.plot(times, results_1[:,0]/volume, label = 'Neutral')
ax.plot(times, results_1[:,1]/volume, label = 'Ion')
ax.legend(loc='best');
ax.set_yscale('log')
